In [1]:
from data import load_data, generate_wrong_proof

train_data, test_data = load_data()

c:\Arjun\Code\ai\ai_debate\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for item in train_data[:100]:
    response = generate_wrong_proof(item["question"], item["answer_correct"]["proof"])
    item['answer_incorrect'] = {"numeric": float(response.split("#")[-1].strip()), "proof": response}

In [12]:
t = load_from_json("data/train_data.json")

In [14]:
for item in t[:100]:
    if item['answer_correct']['numeric'] == item['answer_incorrect']['numeric']:
        print(item['answer_correct']['numeric'], item['answer_incorrect']['numeric'])
        print(item['question'])
        print(item['answer_correct']['proof'])
        print(item['answer_incorrect']['proof'])
        print()

In [7]:
for item in train_data[70:90]:
    print(item['question'])
    print('-')
    print(item['answer_correct']['proof'])
    print('-')
    print(item['answer_incorrect']['proof'])
    # print(generate_wrong_proof(item['question'], item['answer_correct']['proof']))
    print('\n\n\n')

Tim has 30 less apples than Martha, and Harry has half as many apples as Tim. If Martha has 68 apples, how many apples does Harry have?
-
Tim has 68-30 = 38 apples.
Harry has 38/2 = 19 apples.
#### 19
-
Tim has 68-30 = 38 apples.
Harry has 68/2 = 34 apples.
#### 34




At a flea market, Hillary sells handmade crafts for 12 dollars per craft. Today, Hillary sells 3 crafts and is given an extra 7 dollars from an appreciative customer. Later on, Hillary deposits 18 dollars from today's profits into her bank account. How many dollars is Hillary left with after making the deposit?
-
Hillary sells 3 crafts for 12 dollars each, for a total of 3 crafts * $12/craft = $36
She receives an extra 7 dollars from a customer, increasing the total to $36 + $7 = $43
She then deposits 18 dollars in the bank, leaving her with $43 - $18 = $25
#### 25
-
Hillary sells 3 crafts for 12 dollars each, for a total of 3 crafts * $12/craft = $36
She receives an extra 7 dollars from a customer, increasing the total 